In [1]:
import numpy as np
import csv
import string

x_train = np.genfromtxt('processed_train.txt', delimiter=',')
x_train = np.delete(x_train,-1,axis = 1)
x_test = np.genfromtxt('processed_test.txt', delimiter=',')
x_test = np.delete(x_test,-1,axis = 1)
with open('../data/train_labels.csv') as f:
    reader = csv.reader(f, delimiter=",")
    label_train = list(reader)   
del label_train[0]
label_train = np.asarray(label_train)[:,1]
label_train = label_train.tolist()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../data/train_labels.csv')
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

lb_class = LabelEncoder()
obj_df['Category_code']=lb_class.fit_transform(obj_df['Category'])
obj_df[['Category','Category_code']]
y_train = np.asarray(obj_df)[:,1]
y_train = y_train.astype('int')

In [29]:
sample_size = x_train[0:5000].shape[0]
in_dim = x_train[0:5000].shape[1]
out_dim = x_train[0:5000].shape[0]

#NN model and train
def modelbuiler(n_layer, dim_layer, itera=100):
    W = []
    b = []
    W.append(np.random.rand(dim_layer,in_dim))
    b.append(np.zeros((len(W[0]),len(x_train[0:5000]))))
    if n_layer>=3:
        for i in range(1,n_layer-1):
            W.append(np.random.rand(dim_layer,W[i-1].shape[0]))
            b.append(np.zeros((len(W[i]),len(x_train[0:5000]))))
    if n_layer>=2:
        W.append(np.random.randn(31, W[len(W)-1].shape[0]))
        b.append(np.zeros((31,out_dim)))
    model = {}
    
    for iteration in range(0,itera):
        #Forward Prop
        z=[]
        a=[]
        z.append(W[0].dot(x_train[0:5000].T)+b[0])
        a.append(np.tanh(z[0]))
        if n_layer >=3:
            for i in range(1,n_layer-1):
                z.append(W[i].dot(a[i-1])+b[i])
                a.append(np.exp(z[i])/(1+sum(np.exp(z[i]))))
        if n_layer>=2:
            z.append(W[-1].dot(a[-1])+b[-1])
        score = np.exp(z[-1])
        prob = score/np.sum(score,axis=0,keepdims=True)
        yt_prob = np.zeros_like(prob)
        for i in range(0,5000):
            yt_prob[y_train[i],i] = 1
        
        #Back Prop
        delta = []
        dW=[]
        db=[]
        delta.append(prob-yt_prob)
        dW.append(delta[0].dot(a[-1].T))
        db.append(np.sum(delta[0],axis = 0, keepdims=True))
        for i in range(1,n_layer-1):
            db.append(np.sum(delta[i-1],axis = 0, keepdims=True))
            delta.append(W[-i].T.dot(delta[-1])*(1-np.power(a[-i],2)))
            dW.append(delta[i].dot(a[-i-1].T))
        delta.append(W[1].T.dot(delta[-1])*(1-np.power(a[1],2)))
        db.append(np.sum(delta[-1],axis = 0, keepdims=True))
        dW.append(delta[-1].dot(x_train[0:5000]))
        for i in range(0,len(W)):
            W[i]+=-epsilon*dW[-i-1]
            b[i]+=-epsilon*db[-i-1]
        
    model={'W':W, 'b':b}
    return model

#Estimate x
def guess(model, x):
    W, b = model['W'], model['b']
    z=[]
    a=[]
    z.append(W[0].dot(x_train[0:5000].T)+b[0])
    a.append(np.tanh(z[0]))
    if layer_number >=3:
        for i in range(1,layer_number-1):
            z.append(W[i].dot(a[i-1])+b[i])
            a.append(np.exp(z[i])/(1+sum(np.exp(z[i]))))
    if layer_number>=2:
        z.append(W[-1].dot(a[-1])+b[-1])
    score = np.exp(z[-1])
    prob = score/np.sum(score,axis=0,keepdims=True)
    return prob



In [31]:
#Utilize NN Cross Validation
y_true = y_train[5000:10000]
epsilon_valid = [0.00001]
layer_number_valid = [4]
layer_dimension_valid = [30]
error = np.zeros((len(epsilon_valid),len(layer_number_valid),len(layer_dimension_valid)))
for i,epsilon in enumerate(epsilon_valid):
    for j,layer_number in enumerate(layer_number_valid):
        for k,layer_dimension in enumerate(layer_dimension_valid):
            model = modelbuiler(layer_number,layer_dimension)
            y_prob = guess(model, x_train[5000:10000])
            result = np.argmax(y_prob,0)
            diff = y_true-result
            diff = diff.astype(bool).astype(int)
            error[i,j,k] = sum(diff)
            print(error)

[[[4807.]]]


In [32]:
min_error = []
min_error_loc = []
for i in error:
    for j in i:
        min_error.append(min(j))
for i in np.where(error==min(min_error)):
    min_error_loc.append(i[0])
epsilon = epsilon_valid[min_error_loc[0]]
layer_number = layer_number_valid[min_error_loc[1]]
layer_dimension = layer_dimension_valid[min_error_loc[2]]

In [33]:
from sklearn.metrics import classification_report

y_true = y_train[5000:10000]
print(layer_number)
model = modelbuiler(layer_number,layer_dimension)
y_prob = guess(model, x_train[5000:10000])
print(max(y_prob[0]))
result = np.argmax(y_prob,0)
print(classification_report(y_true, result))
print(result)

4
0.030541903563791464
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       169
           1       0.00      0.00      0.00       160
           2       0.00      0.00      0.00       229
           3       0.00      0.00      0.00       287
           4       0.00      0.00      0.00       196
           5       0.00      0.00      0.00       202
           6       0.00      0.00      0.00       113
           7       0.00      0.00      0.00       236
           8       0.00      0.00      0.00       139
           9       0.02      1.00      0.04       110
          10       0.00      0.00      0.00        92
          11       0.00      0.00      0.00       102
          12       0.00      0.00      0.00       106
          13       0.00      0.00      0.00       129
          14       0.00      0.00      0.00       163
          15       0.00      0.00      0.00       138
          16       0.00      0.00      0.00       255
    

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
W=model['W']
for i in W:
    print(i.shape)

(30, 901)
(30, 30)
(30, 30)
(31, 30)


In [7]:
cls = lb_class.inverse_transform(y_pred).tolist()
label = list(range(0,10000))
ID = ['Id']
Cat = ['Category']
ID.extend(label)
Cat.extend(cls)
a=[]
a.append(ID)
a.append(Cat)
print(a)

with open('result_NN.csv', 'w') as f:
    for i in range(0,10001):
        f.write("%s, %s\n" %(a[0][i], a[1][i]))

NameError: name 'y_pred' is not defined